In [9]:
!pip install beautifulsoup4
!pip install requests

In [2]:
!pip list

Package            Version
------------------ -------
attrs              19.3.0
backcall           0.2.0
beautifulsoup4     4.9.1
bleach             3.1.5
colorama           0.4.3
decorator          4.4.2
defusedxml         0.6.0
entrypoints        0.3
importlib-metadata 1.7.0
ipykernel          5.3.2
ipython            7.16.1
ipython-genutils   0.2.0
ipywidgets         7.5.1
jedi               0.17.1
Jinja2             2.11.2
jsonschema         3.2.0
jupyter            1.0.0
jupyter-client     6.1.5
jupyter-console    6.1.0
jupyter-core       4.6.3
MarkupSafe         1.1.1
mistune            0.8.4
nbconvert          5.6.1
nbformat           5.0.7
notebook           6.0.3
numpy              1.19.0
packaging          20.4
pandas             1.0.5
pandocfilters      1.4.2
parso              0.7.0
pickleshare        0.7.5
pip                20.1.1
prometheus-client  0.8.0
prompt-toolkit     3.0.5
Pygments           2.6.1
pyparsing          2.4.7
pyrsistent         0.16.0
python-dateutil  

# Import packages

In [19]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Test Scraping

In [91]:
page_result = requests.get('https://en.wikipedia.org/wiki/Special:Random')

In [92]:
soup = BeautifulSoup(page_result.content, 'html.parser')

In [93]:
# looks like it will be a variable paragraph that we need
# soup.find_all('div', class_ = 'mw-parser-output') #[0].get_text()

In [ ]:
first_paragraph = soup.select('p')[0] #[0].get_text()

<p><i><b>Punk Sucks</b></i> is a <a href="/wiki/California" title="California">California</a> <a href="/wiki/Punk_rock" title="Punk rock">punk rock</a> <a href="/wiki/Compilation_album" title="Compilation album">compilation album</a>, released  by Southern California-based <a href="/wiki/Liberation_Records" title="Liberation Records">Liberation Records</a> in 1995.   
</p>

# use the api idiot

In [97]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11689 sha256=89d2110048f650e69bd2b4ed849e8065a93dd54b100d16005524ad4f16726d5b
  Stored in directory: c:\users\georg\appdata\local\pip\cache\wheels\15\93\6d\5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


## Get the article

In [131]:
import re

In [148]:
page

'United Nations Security Council Resolution 234, was adopted on June 7, 1967, after its appeal to the governments concerned to immediately cease all military activities in the Near East failed. The Council, concerned with the possibility of a broader conflict, demanded that the governments involved discontinue all military activities by 20:00 GMT on June 7, 1967.  The Council also requested that the Secretary-General keep them promptly and currently informed on the situation.\nThe meeting was called by the Soviet Union and the resolution passed unanimously. Jordan and Israel accepted the resolution, provided other parties accepted as well. The following day the United Arab Republic also accepted the ceasefire solution, on condition of reciprocity.'

In [172]:
import wikipedia

term = wikipedia.random(pages=1)
# wiki_wiki = wikipedia.Wikipedia('en')
page = wikipedia.summary(str(term))
# page = wiki_wiki.page('Mathematics')
first_sentence = re.split('[.] ',str(page))[0] + '.'
second_sentence = re.split('[.] ',str(page))[1] + '.'

# print(page)
# print(first_sentence)
# print(second_sentence)
# print(page)

IndexError: list index out of range

## split the question into quiz format

In [168]:
blank = re.split('is|was',str(first_sentence),1)[0]
question = 'BLANK is/was' + re.split('is|was',str(first_sentence), 1)[1]

In [169]:
question

'BLANK is/was an Italian research center for telecommunication based in Torino, the biggest in Italy and one of the most important in Europe.'

In [170]:
second_sentence

'It played a big role internationally especially in the standardization of protocols and technologies in telecommunication: perhaps the most widely well known is the standardization of mp3.'

In [166]:
# print the removed subject
blank

'United Nations Security Council Resolution 234, '

# Get top articles to make it easier

In [173]:
top_articles = requests.get('https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia/all-access/2015/10/10')

In [185]:
# top_articles
# soup = BeautifulSoup(top_articles.content, 'html.parser')
# top_articles.content
# soup.to_dict

In [199]:
import json
data = top_articles.json()

In [209]:
data['items'][0]['articles'][0]['article']

'Main_Page'

In [211]:
# randomly select item from list in top 20% for now
import random
article_number = random.randrange(3,100)

In [212]:
data['items'][0]['articles'][article_number]['article']

'Pan_(2015_film)'

In [ ]:
# works for now, rewrite into functions

# Turn everything into functions now demonstrated it works

In [213]:
def get_top_articles(article_link):
    top_articles = requests.get(str(article_link))
    data = top_articles.json()
    return data

In [214]:
def get_specific_article(article_number, top_articles):
    specific_article = top_articles['items'][0]['articles'][article_number]['article']
    return specific_article

In [215]:
def get_wikipedia_article(specific_article):
    page = wikipedia.summary(str(specific_article))
    return page

In [275]:
def create_question_from_article(page):
    first_sentence = re.split('[. ] ',str(page))[0] + '.'
    second_sentence = re.split('[. ] ',str(page))[1] + '.'
    return first_sentence, second_sentence

In [267]:
def hide_word_and_return_question(first_sentence):
    blank = re.split('( is | was )',str(first_sentence),1)[0]
    question = 'BLANK is/was ' + re.split('( is | was )',str(first_sentence), 1)[2]
    return blank, question

In [287]:
def turn_difficulty_to_range(difficulty):
    # user provides difficulty and we must create a range to select from
    # difficulty 10 should be bottom 10% of articles
    # check integer is between 1-10
    assert (difficulty < 11) & (difficulty > 0)
    
    #calculate bounds
    lower_bound = difficulty * 100
    upper_bound = difficulty * 100 - 100
    return lower_bound, upper_bound

In [476]:
import random
import time
def get_quiz_question(year,month,difficulty):
    # range of choice is difficulty
    # can also adjust date range
    year = str(year)
#     print(month)
    month = '00' + str(month)
    month = month[-2:]
    print(month)
    
    lower_bound, upper_bound = turn_difficulty_to_range(int(difficulty))
    article_number = random.randrange(upper_bound,lower_bound)
    top_link = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia/all-access/{}/{}/all-days'.format(year, month)
    print(top_link)
    articles = get_top_articles(top_link)
    specific_article = get_specific_article(article_number,articles)
    wikipedia_page = get_wikipedia_article(specific_article)
    first_sentence, second_sentence = create_question_from_article(wikipedia_page)
    answer, question = hide_word_and_return_question(first_sentence)
    
    print('Your question is...')
    print(question)
    # Wait for the user input to terminate the program
    

    print('The hint is...')
    print(second_sentence)
    
    # Wait for the user input to terminate the program
    time.sleep(10)
    print('The answer is: ')
    print(answer)
    
    return question, second_sentence, answer

In [437]:
question, second_sentence, answer =  get_quiz_question(year = '2020', month = '06', difficulty = 1)
# difficulty runs 1-10

06
https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia/all-access/2020/06/all-days


In [451]:
question

'BLANK is/was an American political commentator, lawyer, and author, serving as the 31st and current White House Press Secretary since April 2020.\nA graduate of Georgetown University and Harvard University, she began her media career as a producer for Huckabee on Fox News and later worked as a commentator on CNN.'

In [439]:
second_sentence

'In 2017, she was appointed national spokesperson for the Republican National Committee and on April 7, 2020, was appointed as White House Press Secretary in the Trump administration.\nAt the start of the 2016 presidential election, she was critical of then-candidate Trump, calling his remarks about Mexican immigrants "racist" and suggesting it was "inauthentic" to call him a Republican.'

In [440]:
answer

'Kayleigh McEnany (; born April 18, 1988)'

In [417]:
month = '00' + str(1)
print(month)
month[-2:]

001


'01'

In [483]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


slow = interactive(get_quiz_question, {'manual': True}, 
                   difficulty=widgets.IntSlider(min=1, max=10, step=1),
                   month=widgets.IntSlider(min=1, max=12, step=1),
                   year=widgets.IntSlider(min=2016, max=2020, step=1))

In [484]:
slow

interactive(children=(IntSlider(value=2016, description='year', max=2020, min=2016), IntSlider(value=1, descri…

In [488]:
for i in range(10):
    j = i%2
    q = abs(j - 1)
    print(j,q)

0 1
1 0
0 1
1 0
0 1
1 0
0 1
1 0
0 1
1 0


In [ ]:
if q then subtract 1 and take abs value????
what you need is a variable that changes every time unless show answer is clicked
